# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [ ]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

## Q1

Implement the minimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your minimax will be tested using a provided evaluation function on the autograder.

Note about evaluation functions: call the evaluation function in the following manner: `evaluation(board: chess.board, player: bool)`. Think about which player should be passed into the evaluation function; should it be the player who is currently acting (according to the local max/min node) or should it be in accord with the player at the root of the tree?

If you time out during some of the test cases for this problem, consider adding alpha-beta pruning to your implementation.

In [57]:
import chess

def get_minimax_move(b: chess.Board, evaluation, player: bool, ply: int):
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    max_depth = 2 * ply

    def helper(board: chess.Board, depth: int, alpha: float, beta: float):
        # Terminal node
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)

        maximizing = (board.turn == player)

        if maximizing:
            best = float('-inf')
            for move in board.legal_moves:
                board.push(move)
                score = helper(board, depth - 1, alpha, beta)
                board.pop()

                if score > best:
                    best = score
                if best > alpha:
                    alpha = best
                if alpha >= beta:
                    break  # beta cut-off
            return best

        else:
            best = float('inf')
            for move in board.legal_moves:
                board.push(move)
                score = helper(board, depth - 1, alpha, beta)
                board.pop()

                if score < best:
                    best = score
                if best < beta:
                    beta = best
                if beta <= alpha:
                    break  # alpha cut-off
            return best

    best_move = None
    alpha = float('-inf')
    beta = float('inf')

    maximizing_root = (b.turn == player)
    best_value = float('-inf') if maximizing_root else float('inf')

    # Root move ordering: captures first (cheap + effective)
    moves = list(b.legal_moves)
    moves.sort(key=lambda m: b.is_capture(m), reverse=True)

    for move in moves:
        b.push(move)
        value = helper(b, max_depth - 1, alpha, beta)
        b.pop()

        if maximizing_root:
            if value > best_value:
                best_value = value
                best_move = move
            if best_value > alpha:
                alpha = best_value
        else:
            if value < best_value:
                best_value = value
                best_move = move
            if best_value < beta:
                beta = best_value

        if alpha >= beta:
            break  # root cut-off

    return best_move



## Q2

Implement the expectimax algorithm that chooses the best move for a given board position, arbitrary evaluation function, player colour, and depth.

Make sure to write your implementation to be able to use the provided `evaluation()` function when calculating the value of board states. Later on, you'll be writing your own, but for now, your expectimax will be tested using a provided function on the autograder.

You should handle the evaluation function here in the same way as you did in your minimax implementation: `evaluation(board: chess.board, player: bool)`

In [ ]:
def get_expectimax_move(b: chess.Board, evaluation, player: bool, ply: int):   
    """
    This function chooses the best move for the given board position, evaluation function, player, and ply.
    
    Parameters:
    - board: the chess board
    - evaluation: a function that returns a score given a chess board and a player
    - player: the colour of the active player (True -> white, False -> black)
    - ply: the number of move pairs (1 ply = both max AND min) that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """
    max_depth = 2 * ply  # total half-moves

    def helper(board: chess.Board, depth: int):
        if depth == 0 or board.is_game_over():
            return evaluation(board, player)

        maximizing = (board.turn == player)

        legal_moves = list(board.legal_moves)

        if maximizing:
            value = float('-inf')
            for move in legal_moves:
                board.push(move)
                value = max(value, helper(board, depth - 1))
                board.pop()
            return value
        else:
            # enemy node: average over all moves
            if not legal_moves:
                return evaluation(board, player)
            total = 0
            for move in legal_moves:
                board.push(move)
                total += helper(board, depth - 1)
                board.pop()
            return total / len(legal_moves)

    best_move = None
    best_value = float('-inf')

    for move in list(b.legal_moves):
        b.push(move)
        value = helper(b, max_depth - 1)
        b.pop()

        if value > best_value:
            best_value = value
            best_move = move

    return best_move

## Part 2

You will need to complete the implementation of your minimax function before continuing, as the puzzles that follow depend on its correctness.

In the previous questions, you wrote your implementation of minimax and tested it using a hidden evaluation function we wrote for you. Now, you are tasked with developing an original evaluation function `get_position_score` that will be used in conjunction with your minimax function. 

There are several chess puzzles below (drawn from `lichess.org`). Iterate on your evaluation function so that it can solve each puzzle as you come to it. During grading for each puzzle, you must solve the visible case as well as one hidden case to receive full credit. A general solution should be able to solve both.

In [ ]:
def get_position_score(board: chess.Board, player: bool):   
    """
    This function returns a score for a board position, from the given player's point of view.
    A higher score should be more favourable than a low score.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    A numerical value representing the score of the board position from player's point of view.
    """
    piece_values = {
        chess.PAWN: 2,
        chess.KNIGHT: 2.5,
        chess.BISHOP: 2.5,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }

    score = 0
    pieces = board.piece_map()

    # encourage moving pawns across if no opposing pawns
    def _is_passed_pawn(square, color):
        file = chess.square_file(square)
        rank = chess.square_rank(square)
        step = 1 if color == chess.WHITE else -1
        # check the ranks in front of the pawn
        for r in range(rank + step, 8 if color == chess.WHITE else -1, step):
            for f in range(max(0, file - 1), min(7, file + 1) + 1):
                sq = chess.square(f, r)
                p = board.piece_at(sq)
                if p and p.piece_type == chess.PAWN and p.color != color:
                    return False # enemy pawn blocks the path
        return True # no enemy pawn blocks the pawn's advancement

    # check if pawn is unblocking a friendly rook
    def _pawn_unblocks_rook(square, color):
        file = chess.square_file(square)
        rank = chess.square_rank(square)
        # Check same file
        for r in range(0, 8):
            if r == rank:
                continue
            sq = chess.square(file, r)
            p = board.piece_at(sq)
            if p and p.piece_type == chess.ROOK and p.color == color:
                return True
        # Check same rank
        for f in range(0, 8):
            if f == file:
                continue
            sq = chess.square(f, rank)
            p = board.piece_at(sq)
            if p and p.piece_type == chess.ROOK and p.color == color:
                return True
        return False

    # iterate through pieces on the board
    for sq, piece in pieces.items():
        value = piece_values[piece.piece_type]

        # base score: positive for the player, negative for the opponent
        score += value if piece.color == player else -value

        if piece.piece_type == chess.PAWN:
            # Passed pawn bonus
            if _is_passed_pawn(sq, piece.color):
                score += 2.0 if piece.color == player else -2.0

            # Advancing pawn bonus: encourage pushing pawns forward
            rank = chess.square_rank(sq)
            pawn_adv = rank / 7 if piece.color == chess.WHITE else (7 - rank) / 7
            score += 0.5 * pawn_adv if piece.color == player else -0.5 * pawn_adv

            # Rook-unblocking bonus: encourages freeing rooks
            if _pawn_unblocks_rook(sq, piece.color):
                score += 2.0 if piece.color == player else -2.0

        # Minor piece centralization: we like knights/bishops near the center
        if piece.piece_type in {chess.KNIGHT, chess.BISHOP}:
            rank = chess.square_rank(sq)
            file = chess.square_file(sq)
            dist_to_center = ((3.5 - file) ** 2 + (3.5 - rank) ** 2) ** 0.5
            central_bonus = 0.01 * (4 - dist_to_center)
            score += central_bonus if piece.color == player else -central_bonus

        # King activity: we like the king in the center in endgame
        if piece.piece_type == chess.KING:
            rank = chess.square_rank(sq)
            file = chess.square_file(sq)
            dist_to_center = ((3.5 - file) ** 2 + (3.5 - rank) ** 2) ** 0.5
            score += 0.05 * dist_to_center if piece.color == player else -0.05 * dist_to_center

    return score

### Q3

Given the puzzle below, modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, it will return a move resulting in checkmate.

<img src="images/asgn2%20q3%20puzzle.png" alt="assignment 2 question 3 puzzle figure" width="400"/>

In [82]:
# Feel free to use this code to test your solution to the puzzle above.
puzzle1_board = chess.Board("1rr5/p1p2Rpp/2Qpk3/4n1q1/4P3/8/PPP3PP/R6K")
move = get_minimax_move(puzzle1_board, get_position_score, True, 1)
print(f"Move: {move}")

Move: c6d5


#### Setting up an Engine

To generate an engine score, first you will need to download the chess engine 'Stockfish' from https://stockfishchess.org/download/.

Once that's done, you may find this docs page useful in setting it up: https://python-chess.readthedocs.io/en/latest/engine.html

Here, the engine score is given in 'Centipawns.' Modify the code below to match your own system and to confirm you have the engine set up correctly.

Note: you may notice that the scores returned by the engine are not consistent. That is, the same position may return a slightly different score. However, because the difference is small, it should not be an issue as the general trends hold true and the puzzles here tend to have an unambiguous best answer. As a failsafe, if your implementation returns the move we're looking for, you'll get full credit regardless of whatever score the engine returns.

In [42]:
import chess.engine
engine = chess.engine.SimpleEngine.popen_uci(r"/home/username/code/cse240/Assignment2/stockfish/stockfish-windows-x86-64-avx2.exe") 

# A completely standard board. White starts at a slight advantage, having first move.
board1 = chess.Board()
info1 = engine.analyse(board1, chess.engine.Limit(time=0.1))
print(f"Score: {info1['score'].white()}")

# A standard board, but White is missing both rooks. Naturally, White should have the disadvantage now.
board2 = chess.Board("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/1NBQKBN1")
info2 = engine.analyse(board2, chess.engine.Limit(time=0.1))
print(f"Score: {info2['score'].white()}")

engine.quit()

Score: +47
Score: -678


#### Q4
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q4%20puzzle.png" alt="assignment 2 question 34 puzzle figure" width="400"/>

In [78]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle2_board = chess.Board("8/1r3k2/2r1ppp1/8/5PB1/4P3/4PK2/5R2")
engine = chess.engine.SimpleEngine.popen_uci(r"/home/username/code/cse240/Assignment2/stockfish/stockfish-windows-x86-64-avx2.exe") 
pre_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle2_board, get_position_score, True, 3)
print(f"Move: {move}")
puzzle2_board.push(move)

post_move_info = engine.analyse(puzzle2_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: g4f3
Pre-Move Score: 0
Post-Move Score: +1


#### Q5
Given the puzzle below (White to move), modify the function `get_position_score` such that, when used by your minimax function as seen in the cell below, will return a good continuation to the game position. 

You cannot use the engine itself in your evaluation function, but you may find it helpful as you figure out what could make for good features in your evaluation function. Also recommended are the chess position analysis tools available online, such as `https://lichess.org/analysis/standard` to help you get some intuition about what sort of actions may be desirable. What kind of factors would you imagine go into scoring a chess board position? Try to implement some of those in your `get_position_score` function. 

<img src="images/asgn2%20q5%20puzzle.png" alt="assignment 2 question 5 puzzle figure" width="400"/>

In [86]:
# As above, so below; feel free to use this code to test your solution to the puzzle above.
puzzle3_board = chess.Board("1k6/2b2p2/2p1p3/1pP2p2/1P1P1P2/8/2N1P3/1K6")
engine = chess.engine.SimpleEngine.popen_uci(r"/home/username/code/cse240/Assignment2/stockfish/stockfish-windows-x86-64-avx2.exe")
pre_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))

move = get_minimax_move(puzzle3_board, get_position_score, True, 1)
print(f"Move: {move}")
puzzle3_board.push(move)

post_move_info = engine.analyse(puzzle3_board, chess.engine.Limit(time=0.1))
engine.quit()
print(f"Pre-Move Score: {pre_move_info['score'].white()}")
print(f"Post-Move Score: {post_move_info['score'].white()}")

Move: c2e3
Pre-Move Score: +43
Post-Move Score: -136
